#Reglas del Juego:
1. El tablero será de tamaño 10x10.
2. Cada jugador tendrá 5 barcos de diferentes tamaños:
- portaaviones (tamaño 5)
- acorazado (tamaño 4)
- crucero (tamaño 3)
- submarino (tamaño 3)
- destructor (tamaño 2)
3. Los barcos podrán colocarse tanto en posición horizontal como vertical.
4. Durante el turno de un jugador, este deberá elegir una coordenada para atacar (fila y columna). Si acierta, el barco atacado recibirá un golpe. Si se destruyen todas las partes de un barco, este se considerará hundido.
5. El juego termina cuando un jugador logra hundir todos los barcos del oponente.

In [1]:
print("Bienvenido al juego de Hundir la Flota")

Bienvenido al juego de Hundir la Flota


In [2]:
import numpy as np

In [3]:
class Barco:
    def __init__(self, nombre: str, tamaño: int, posicion: tuple, orientacion: str,golpes=0):
        self.nombre = nombre
        self.tamaño = tamaño
        self.posicion = posicion  # (fila, columna)
        self.orientacion = orientacion  # "horizontal" o "vertical"
        self.golpes = golpes    # Inicializa los golpes en el valor dado por defecto en 0


    def recibir_golpe(self):
         # Aumenta el contador de golpes y verifica si el barco está hundido
        self.golpes+=1
        print(f"{self.nombre} tocado!")  
        if self.esta_hundido():
            print(f"{self.nombre} hundido!")  
            return True
        return False

    def esta_hundido(self):
        # Retorna True si el barco ha recibido tantos golpes como su tamaño y False si no
        return self.golpes==self.tamaño
    
    def estado_barco(self):
        # Retorna el símbolo del estado del barco
        if self.esta_hundido():
            return "X"  # Barco hundido
        elif self.golpes > 0:
            return "O"  # Barco tocado
        return "B"  # Barco intacto
    

In [ ]:
import numpy as np
class Tablero:
    def __init__(self, tamaño: int = 10):
        # Inicializa el tablero como una matriz 10x10 llena de espacios " "
        self.tamaño = tamaño
        self.tablero = np.full((tamaño, tamaño), " ")
        self.barcos = []  # Lista para almacenar los barcos colocados

    def colocar_barco(self, barco: Barco):
        # Comprobar si el barco cabe en la posición dada sin colisionar
        if self.puede_colocar_barco(barco):
            if barco.orientacion == "horizontal":
                for i in range(barco.tamaño):
                    self.tablero[barco.posicion[0], barco.posicion[1] + i] = barco.estado_barco()
            else:  # Orientación vertical
                for i in range(barco.tamaño):
                    self.tablero[barco.posicion[0] + i, barco.posicion[1]] = barco.estado_barco()
            # Añadir el barco a la lista de barcos en el tablero
            self.barcos.append(barco)
        
            

    def puede_colocar_barco(self, barco: Barco):
        # Comprobación de límites y colisiones
        if barco.orientacion == "horizontal":
             # Verificar si el barco se sale del borde derecho del tablero
            if barco.posicion[1] + barco.tamaño > self.tamaño:
                print(f"El barco '{barco.nombre}' no cabe horizontalmente desde la columna {barco.posicion[1]}. Se sale del tablero.")
                return False  # El barco se sale del tablero
            # Verificar si hay colisión en la posición horizontal
            for i in range(barco.tamaño):
                if self.tablero[barco.posicion[0], barco.posicion[1] + i] == "B":
                    print(f"El barco '{barco.nombre}' colisiona con otro barco en la posición ({barco.posicion[0]}, {barco.posicion[1] + i}).")
                    return False  # Hay colisión con otro barco
        else:  # Orientación vertical
         # Verificar si el barco se sale del borde inferior del tablero
            if barco.posicion[0] + barco.tamaño > self.tamaño:
                print(f"El barco '{barco.nombre}' no cabe verticalmente desde la fila {barco.posicion[0]}. Se sale del tablero.")
                return False
        
            # Verificar si hay colisión en la posición vertical
            for i in range(barco.tamaño):
                if self.tablero[barco.posicion[0] + i, barco.posicion[1]] == "B":
                    print(f"El barco '{barco.nombre}' colisiona con otro barco en la posición ({barco.posicion[0] + i}, {barco.posicion[1]}).")
                    return False
                    
        # Si no hay problemas, retornar True
        return True


    def recibir_ataque(self, fila, columna):
        # Verificar si las coordenadas están dentro del tablero
        if fila < 0 or fila >= self.tamaño or columna < 0 or columna >= self.tamaño:
            print("Coordenadas fuera del tablero.")
            return False
        
        
        for barco in self.barcos:
            # Verificar si el barco está en la posición atacada
            if barco.orientacion == "horizontal":
                if fila == barco.posicion[0] and barco.posicion[1] <= columna < barco.posicion[1] + barco.tamaño:
                    if barco.recibir_golpe():
                        self.tablero[fila, columna] = "X"  # Marcar como hundido
                        if self.todos_barcos_hundidos():
                            break
                    else:
                        self.tablero[fila, columna] = "O"  # Marcar como tocado
                    return True
            else:  # Orientación vertical
                if columna == barco.posicion[1] and barco.posicion[0] <= fila < barco.posicion[0] + barco.tamaño:
                    if barco.recibir_golpe():
                        self.tablero[fila, columna] = "X"  # Marcar como hundido
                        if self.todos_barcos_hundidos():
                            break
                    else:
                        self.tablero[fila, columna] = "O"  # Marcar como tocado
                    return True

        print("Agua")  # No se encontró ningún barco en la posición
        self.tablero[fila, columna] = "A" # Si no se ha tocado ningún barco, marcar como agua
        return False 


    def mostrar_tablero(self):
        for fila in self.tablero:
            fila_mostrada = []
            for casilla in fila:
                fila_mostrada.append(casilla)  # Mostrar el estado (B, O, X o espacio)
            print(" | ".join(fila_mostrada))  # Usar | para separar las casillas

        


    def todos_barcos_hundidos(self):
        for barco in self.barcos:
            if not barco.esta_hundido():
                return False
        print('Enhorabuena ,¡¡ Has ganado!!,todos los barcos han sido hundidos.')    
        return True
        

In [5]:
import random
class Jugador:
    def __init__(self, nombre: str, tablero:Tablero):
        self.nombre = nombre
        self.tablero = tablero  # Cada jugador tiene su propio tablero

    def atacar(self, oponente, fila: int, columna: int):
        # Realizar el ataque al tablero del oponente

        if oponente.tablero.recibir_ataque(fila, columna):
            ataque_exitoso = True  # El ataque fue exitoso
           
        else:
            ataque_exitoso = False  # El ataque no fue exitoso
        return ataque_exitoso
    
    def generar_ataque_aleatorio(self):
        # Generar coordenadas aleatorias para atacar
        fila = random.randint(0, 9)
        columna = random.randint(0, 9)
        return fila, columna    

In [6]:
# barcos_jugador1 = Barco(nombre="Portaaviones", tamaño=5, posicion=(0, 0), orientacion="horizontal")
            
# barcos_jugador2 = Barco(nombre="Portaaviones", tamaño=5, posicion=(8, 0), orientacion="horizontal")

# tablero_jugador1=Tablero()
# tablero_jugador2=Tablero()

# tablero_jugador1.colocar_barco(barcos_jugador1)
# tablero_jugador2.colocar_barco(barcos_jugador2)

# print("\nTablero del Jugador 1:")
# tablero_jugador1.mostrar_tablero()
# print("\nTablero del Jugador 2:")
# tablero_jugador2.mostrar_tablero()

# jugador1 = Jugador("Jugador 1", tablero_jugador1)
# jugador2 = Jugador("Ordenador", tablero_jugador2)


# juego=Juego(jugador1, jugador2)

In [7]:
# jugador1.atacar(jugador2, 8, 1)
# jugador1.atacar(jugador2, 8, 2)
# jugador1.atacar(jugador2, 8, 3)
# jugador1.atacar(jugador2, 8, 4)

In [8]:
# tablero_jugador2.mostrar_tablero()

In [ ]:
import random
import numpy as np

class Juego:
    def __init__(self):
        # Solicitar el nombre del Jugador 1
        nombre_jugador1 = input("Ingrese el nombre del Jugador 1: ")

        # Inicializa los tableros y jugadores
        self.tablero_jugador1 = Tablero()
        self.tablero_jugador2 = Tablero()
        
        self.jugador1 = Jugador(nombre=nombre_jugador1, tablero=self.tablero_jugador1)
        self.jugador2 = Jugador(nombre="Ordenador", tablero=self.tablero_jugador2)
        
        # Configura barcos para ambos jugadores
        self.configurar_barcos()

        # Establece el jugador actual (Jugador 1 comienza)
        self.jugador_actual = self.jugador1
        self.oponente = self.jugador2

    def alternar_turno(self):
        # Alternar el jugador actual
        if self.jugador_actual == self.jugador1:
            self.jugador_actual = self.jugador2
            self.oponente = self.jugador1
        else:
            self.jugador_actual = self.jugador1
            self.oponente = self.jugador2

    def configurar_barcos(self):
        # Configura barcos para el jugador 1
        barcos_jugador1 = [
            Barco(nombre="Portaaviones", tamaño=5, posicion=(0, 0), orientacion="horizontal")
            # Barco(nombre="Acorazado", tamaño=4, posicion=(0, 0), orientacion="horizontal"),
            # Barco(nombre="Crucero", tamaño=3, posicion=(0, 0), orientacion="horizontal"),
            # Barco(nombre="Submarino", tamaño=3, posicion=(0, 0), orientacion="horizontal"),
            # Barco(nombre="Destructor", tamaño=2, posicion=(0, 0), orientacion="horizontal"),
        ]

        for barco in barcos_jugador1:
            while True:
                print(f"\nColoca tu barco {barco.nombre} de tamaño {barco.tamaño}.")
                fila = int(input("Ingrese la fila (0-9): "))
                columna = int(input("Ingrese la columna (0-9): "))
                orientacion = input("Ingrese la orientación (horizontal/vertical): ").strip().lower()

                # Crear barco con la posición ingresada
                barco.posicion = (fila, columna)
                barco.orientacion = orientacion

                # Intentar colocar el barco
                if self.tablero_jugador1.puede_colocar_barco(barco):
                    self.tablero_jugador1.colocar_barco(barco)
                    break
                else:
                    print("No se puede colocar el barco. Intente nuevamente.")

        # Configura barcos para el ordenador de forma aleatoria
        self.colocar_barcos_aleatoriamente(self.tablero_jugador2)

    def colocar_barcos_aleatoriamente(self, tablero:Tablero):
        barcos_jugador2 = [
            Barco(nombre="Portaaviones", tamaño=5, posicion=(0, 0), orientacion="horizontal")
            # Barco(nombre="Acorazado", tamaño=4, posicion=(0, 0), orientacion="horizontal"),
            # Barco(nombre="Crucero", tamaño=3, posicion=(0, 0), orientacion="horizontal"),
            # Barco(nombre="Submarino", tamaño=3, posicion=(0, 0), orientacion="horizontal"),
            # Barco(nombre="Destructor", tamaño=2, posicion=(0, 0), orientacion="horizontal"),
        ]

        for barco in barcos_jugador2:
            colocado = False
            while not colocado:
                fila = random.randint(0, 9)
                columna = random.randint(0, 9)
                orientacion = random.choice(["horizontal", "vertical"])
                barco.posicion = (fila, columna)
                barco.orientacion = orientacion
                
                if tablero.puede_colocar_barco(barco):
                    tablero.colocar_barco(barco)
                    colocado = True

    def iniciar_juego(self):
        print(f"¡Bienvenido a Hundir la Flota, {self.jugador1.nombre}!")
        while True:
            # Realiza el turno actual y verifica si el oponente ha perdido
            turno_continua = self.turno()

            if self.oponente.tablero.todos_barcos_hundidos():
                print(f"¡Felicidades {self.jugador_actual.nombre}, has ganado!@")
                break
            
            # Cambia de turno solo si el ataque falló (turno_continua == False)
            if not turno_continua:
                self.alternar_turno()

    def turno(self):
        print(f"\nEs el turno de {self.jugador_actual.nombre}..")
        self.oponente.tablero.mostrar_tablero()  # Mostrar el tablero del Defensor
        sigue=False
        while True:
            if self.jugador_actual.nombre == "Ordenador":
                # print("\nTablero del Jugador 1:")
                # self.tablero_jugador1.mostrar_tablero()  # Mostrar el tablero del Jugador 1
                fila, columna = self.jugador_actual.generar_ataque_aleatorio()
                
            else:
                # print(f"Turno de {atacante.nombre}.")
                # print("\nTablero del Ordenador:")
                # self.tablero_jugador2.mostrar_tablero()  # Mostrar el tablero del Ordenador
                fila = int(input("Ingrese la fila (0-9) para atacar: "))
                columna = int(input("Ingrese la columna (0-9) para atacar: "))
                
            
            if 0 <= fila < 10 and 0 <= columna < 10: #Comprobamos que las coordenadas estén dentro del tablero
                # if self.oponente.tablero[fila,columna] not in ["X", "O"]:    # Comprobamos que la casilla no esté ocupada (hay que mirar por que no coge self.oponente.tablero[fila,columna] )
                    
                # Realiza el ataque y muestra el tablero actualizado
                
                sigue = self.jugador_actual.atacar(self.oponente, fila, columna)  # Llama a atacar y usa el retorno
                print('Tablero oponente.')
                self.oponente.tablero.mostrar_tablero()  # Muestra el tablero después del ataque
                break  # Sale del ciclo

                # else: 
                #     print('Cordenadas con casillas ocupadas, por favor introduzca nuevas coordenadas.')
            else:        
                print('Cordenadas fuera del rango del tablero, por favor introduzca nuevas coordenadas.')
        return sigue

In [ ]:
juego=Juego()
juego.iniciar_juego()


Coloca tu barco Portaaviones de tamaño 5.
¡Bienvenido a Hundir la Flota, Deni!

Es el turno de Deni..
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
B | B | B | B | B |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
tablero oponente
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
B | B | B | B | B |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
Portaaviones tocado!
tablero oponente.
  |   |   |   |   |   |   |   |   |  
  |   |   |   |   |   |   |   |   |  
  |  

# Mejoras a realizar:
- Ques se compruebe que la casilla no esté ocupada antes de realizar el ataque.
- Si se ingresa mal las coordenadas (en vez de int un string ), o la orientación (escribir mal 'horizontal' o 'vertical'[esta contemplado ya que no de error si son mayúsculas o minúsculas]), se genera un error.
- Optimizar la forma de pintar el tablero.